## Example: How to use the ilastik Python API for Pixel Classification with multiprocessing

The first version of the ilastik API allows you to predict your data with a previously trained ilastik project directly from Python.

This example shows how to process multiple images in parallel and helps measure speedups

Use data courtesy of the flyem team

In [ ]:
# Add your imports here, e.g. for loading and preprocessing data
from concurrent import futures
from pathlib import Path
from time import perf_counter

import imageio
import numpy
import psutil
import requests
import tqdm
from xarray import DataArray

from ilastik.experimental.api import from_project_file

In [ ]:
img_path = Path("./imgs")
img_path.mkdir(exist_ok=True)
base_url = "https://data.ilastik.org/flyem"

n_images = 32  # can go up to 99

In [ ]:
# download the images
for i in tqdm.tqdm(range(n_images)):
    r = requests.get(f"{base_url}/iso.26{i:02d}.png")
    (img_path / f"iso.26{i:02d}.png").write_bytes(r.content)

In [ ]:
# naive approach, process each individual file
project_file = "pc.ilp"
pipeline = from_project_file(project_file)

predictions_naive = []
t0 = perf_counter()
for i in tqdm.tqdm(range(n_images)):
    image = DataArray(imageio.imread(img_path / f"iso.26{i:02d}.png"), dims=("y", "x"))
    prediction = pipeline.predict(image)
    predictions_naive.append(prediction)

t_naive = perf_counter() - t0
print(f"Naive execution took {t_naive:.2f} s")

In [ ]:
pipeline = from_project_file("pc.ilp")

def predict_ilp(image_file):

    image = DataArray(imageio.imread(image_file), dims=("y", "x"))
    prediction = pipeline.predict(image)
    print(f"done {image_file}")
    return prediction


# multiprocessing
t0 = perf_counter()
with futures.ProcessPoolExecutor(max_workers=psutil.cpu_count()) as pool:
    futs = pool.map(predict_ilp, [img_path / f"iso.26{i:02d}.png" for i in range(n_images)])
    predictions_mp = [x for x in futs]
t_mp = perf_counter() - t0
print(f"Multiprocessing execution with {psutil.cpu_count()} workers took {t_mp} s")

In [ ]:
# sanity check
for a, b in zip(predictions_naive, predictions_mp):
    numpy.testing.assert_array_equal(a, b)